<a href="https://colab.research.google.com/github/ADUmesh/Video-Summarization-using-Encoder-and-Decoder-Techniques/blob/main/Video_Summerization_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Handling :**
In the following code blocks, we have read the h5 dataset (TVSum) and splitted it into train and test set.

In [ ]:
# Importing essential libraries
import h5py
import numpy as np
import tensorflow.keras as Keras
from sklearn.model_selection import train_test_split

In [ ]:
# Mount your drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Defining a generator object to handle dataset
import h5py
import numpy as np
import tensorflow.keras as Keras
from sklearn.model_selection import train_test_split

class DataGenerator(Keras.utils.Sequence):

    def __init__(self,dataset,batch_size=5,shuffle=False):
        '''Initialise the dataset'''
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        '''Returns length of the dataset'''
        return int(np.floor(len(self.dataset)/self.batch_size))

    def __getitem__(self,index):
        '''Returns items of given index'''
        indexes = self.indices[index * self.batch_size : (index+1) * self.batch_size]
        feature, label = self.__data_generation(indexes)

        return feature, label

    def __data_generation(self,indexes):
        '''Generates data from given indices'''
        feature = np.empty((self.batch_size,320,1024))
        label = np.empty((self.batch_size,320,1))

        for i in range(len(indexes)):
            feature[i,] = np.array(self.dataset[indexes[i]][0])
            label[i,] = np.array(self.dataset[indexes[i]][1]).reshape(-1,1)

        return feature,label

    def on_epoch_end(self):

        self.indices = np.arange(len(self.dataset))
        if self.shuffle == True:
            np.random.shuffle(self.indices)

In [ ]:
# Defining a class to read the h5 dataset
class DatasetMaker(object):

    def __init__(self,data_path):
        '''Read file from defined path'''

        self.data_file = h5py.File(data_path)

    def __len__(self):
        '''Returns length of the file'''

        return len(self.data_file)

    def __getitem__(self,index):
        '''Returns feature, label and index of varoius keys'''

        index += 1
        video = self.data_file['video_'+str(index)]
        feature = np.array(video['feature'][:])
        label = np.array(video['label'][:])

        return feature,label,index

# Defining a function to read and split the dataset into train and test
def get_loader(path, batch_size=5):
    '''Takes file path as argument and returns train and test set'''

    dataset = DatasetMaker(path)
    train_dataset, test_dataset = train_test_split(dataset, test_size = 0.2, random_state = 123)
    train_loader = DataGenerator(train_dataset)

    return train_dataset, train_loader, test_dataset

In [ ]:
# Loading and splitting the dataset
train_dataset, train_loader, test_dataset = get_loader('/content/drive/MyDrive/suvidha internship/Resources/Submission/fcsn_tvsum.h5')

In [ ]:
# Getting an overview of train set
train_loader.__getitem__(1)

(array([[[0.33214834, 0.36458752, 0.34766391, ..., 0.42054603,
          0.38009831, 0.40780613],
         [0.3677392 , 0.32178515, 0.3589496 , ..., 0.38658553,
          0.40404913, 0.41493717],
         [0.32732472, 0.34487578, 0.34561759, ..., 0.39767522,
          0.40643126, 0.41305968],
         ...,
         [0.33563241, 0.37010661, 0.36633509, ..., 0.45292339,
          0.39594346, 0.37678552],
         [0.33977899, 0.36665431, 0.36543891, ..., 0.45478585,
          0.39663646, 0.37431845],
         [0.33928111, 0.36744118, 0.3657186 , ..., 0.45513114,
          0.39545387, 0.37432   ]],
 
        [[0.35793796, 0.32084152, 0.37723896, ..., 0.41072085,
          0.3839736 , 0.41116855],
         [0.36485991, 0.33529118, 0.37920213, ..., 0.41223219,
          0.3917633 , 0.43476462],
         [0.35199738, 0.36249325, 0.37726289, ..., 0.42207533,
          0.36644816, 0.38966513],
         ...,
         [0.3703391 , 0.3461917 , 0.37459537, ..., 0.40429765,
          0.33990452, 0.

In [ ]:
# Getting an overview of test set
test_dataset[1]

(array([[0.35971966, 0.3486733 , 0.36395055, ..., 0.42930636, 0.4150221 ,
         0.41987327],
        [0.36696154, 0.37052545, 0.37211522, ..., 0.4189313 , 0.4207554 ,
         0.39118388],
        [0.358309  , 0.37929296, 0.34820387, ..., 0.39622623, 0.39785638,
         0.3821973 ],
        ...,
        [0.3435912 , 0.33795008, 0.35553935, ..., 0.41903496, 0.40541124,
         0.42342108],
        [0.34438112, 0.347014  , 0.37080073, ..., 0.42950776, 0.3868425 ,
         0.36775845],
        [0.35290626, 0.35092926, 0.37546453, ..., 0.4372162 , 0.37105322,
         0.39629552]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
        1., 1.

# **Model Training and Testing :**
In this section, we have defined various functions to handle the algorithm in order and trained the model and tested it over the test set.

In [ ]:
import numpy as np

def knapsack(s, w, c): #shot, weights, capacity


    shot = len(s) + 1 #number of shots
    cap = c + 1 #capacity threshold

    #matching the modified size by adding 0 at 0th index
    s = np.r_[[0], s] #adding 0 at 0th index (concatinating)
    w = np.r_[[0], w] #adding 0 at 0th index (concatinating)

    #Creating and Filling Dynamic Programming Table with zeros in shot x cap dimensions
    dp = [] #creating empty list or table
    for j in range(shot):
        dp.append([]) #s+1 rows
        for i in range(cap):
            dp[j].append(0) #c+1 columns

    #Started filling values from (2nd row, 2nd column) till (shot X cap) and keeping the values for 0th indexes as 0
    #following dynamic programming approach to fill values
    for i in range(1,shot):
        for j in range(1,cap):
            if w[i] <= j:
                dp[i][j] = max(s[i] + dp[i-1][j-w[i]], dp[i-1][j])
            else:
                dp[i][j] = dp[i-1][j]

    #choosing the optimal pair of keyshots
    choice = []
    i = shot - 1
    j = cap - 1
    while i > 0 and j > 0:
        if dp[i][j] != dp[i-1][j]: #starting from last element and going further
            choice.append(i-1)
            j = j - w[i]
            i = i - 1
        else:
            i = i - 1

    return dp[shot-1][cap-1], choice

In [ ]:
import numpy as np

def eval_metrics(y_pred, y_true):
    '''Returns precision, recall and f1-score of given prediction and true value'''
    overlap = np.sum(y_pred * y_true)
    precision = overlap / (np.sum(y_pred) + 1e-8)
    recall = overlap / (np.sum(y_true) + 1e-8)
    if precision == 0 and recall == 0:
        fscore = 0
    else:
        fscore = 2 * precision * recall / (precision + recall)

    return [precision, recall, fscore]

In [ ]:
def select_keyshots(video_info, pred_score):
    '''Returns predicted scores(upsampled), selected keyshots indices, predicted summary of given video'''
    vidlen = video_info['length'][()] # Getting video length
    cps = video_info['change_points'][:] # Getting change points (shot changes)
    weight = video_info['n_frame_per_seg'][:] # Getting number of frames per shot info
    pred_score = np.array(pred_score)
    pred_score = upsample(pred_score, vidlen)
    pred_value = np.array([pred_score[cp[0]:cp[1]].mean() for cp in cps])
    _, selected = knapsack(pred_value, weight, int(0.2 * vidlen)) # Knapsacking 20% of the video as summary
    selected = selected[::-1]
    key_labels = np.zeros((vidlen,))
    for i in selected:
        key_labels[cps[i][0]:cps[i][1]] = 1

    return pred_score.tolist(), selected, key_labels.tolist()

In [ ]:
def upsample(down_arr, vidlen):
    '''Upsamples a given predicted score array to the size of video length'''
    up_arr = np.zeros(vidlen)
    ratio = vidlen // 320
    l = (vidlen - ratio * 320) // 2
    i = 0
    while i < 320:
        up_arr[l:l+ratio] = np.ones(ratio, dtype = int) * down_arr[0][i]
        l += ratio
        i += 1

    return up_arr

In [ ]:
import numpy as np
import json
import os
from tqdm import tqdm, trange
import h5py
from prettytable import PrettyTable

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, TimeDistributed, Concatenate
from tensorflow.keras.layers import Attention
from tensorflow.keras import Input, Model

def evaluate_train(epoch_i):
    '''Evaluates the model for given epoch on test dataset'''
    out_dict = {}
    eval_arr = []
    table = PrettyTable()
    table.title = 'Evaluation Result of epoch {} on Training Set'.format(epoch_i)
    table.field_names = ['ID', 'Precision', 'Recall', 'F-Score']
    table.float_format = '1.5'

    with h5py.File('/content/drive/MyDrive/suvidha internship/Resources/Submission/fcsn_tvsum.h5') as data_file:
        for feature, label, index in tqdm(train_dataset, desc = 'Evaluate', ncols = 90, leave = False):

            pred_score = model.predict(feature.reshape(-1,320,1024)) # Predicting for 'index' video
            video_info = data_file['video_'+str(index)]
            pred_score, pred_selected, pred_summary = select_keyshots(video_info, pred_score) # Getting predictions of shot level
            true_summary_arr = video_info['user_summary'][:] # True summary of the video
            eval_res = [eval_metrics(pred_summary, true_summary) for true_summary in true_summary_arr] # Getting evaluation metrics of that particular video
            eval_res = np.mean(eval_res, axis = 0).tolist() # Averaging out the metrics

            eval_arr.append(eval_res)
            table.add_row([index] + eval_res)

            out_dict[str(index)] = {
            'pred_score' : pred_score,
            'pred_selected' : pred_selected,
            'pred_summary' : pred_summary
            }
    eval_mean = np.mean(eval_arr, axis = 0).tolist()
    table.add_row(['mean'] + eval_mean)
    tqdm.write(str(table))

In [ ]:
import numpy as np
import json
import os
from tqdm import tqdm, trange
import h5py
from prettytable import PrettyTable

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, TimeDistributed, Concatenate
from tensorflow.keras.layers import Attention
from tensorflow.keras import Input, Model

def evaluate_test(epoch_i):
    '''Evaluates the model for given epoch on test dataset'''
    out_dict = {}
    eval_arr = []
    table = PrettyTable()
    table.title = 'Evaluation Result of epoch {} on Testing Set'.format(epoch_i)
    table.field_names = ['ID', 'Precision', 'Recall', 'F-Score']
    table.float_format = '1.5'

    with h5py.File('/content/drive/MyDrive/suvidha internship/Resources/Submission/fcsn_tvsum.h5') as data_file:
        for feature, label, index in tqdm(test_dataset, desc = 'Evaluate', ncols = 90, leave = False):

            pred_score = model.predict(feature.reshape(-1,320,1024)) # Predicting for 'index' video
            video_info = data_file['video_'+str(index)]
            pred_score, pred_selected, pred_summary = select_keyshots(video_info, pred_score) # Getting predictions of shot level
            true_summary_arr = video_info['user_summary'][:] # True summary of the video
            eval_res = [eval_metrics(pred_summary, true_summary) for true_summary in true_summary_arr] # Getting evaluation metrics of that particular video
            eval_res = np.mean(eval_res, axis = 0).tolist() # Averaging out the metrics

            eval_arr.append(eval_res)
            table.add_row([index] + eval_res)

            out_dict[str(index)] = {
            'pred_score' : pred_score,
            'pred_selected' : pred_selected,
            'pred_summary' : pred_summary
            }
    eval_mean = np.mean(eval_arr, axis = 0).tolist()
    table.add_row(['mean'] + eval_mean)
    tqdm.write(str(table))

In [ ]:
# Baseline implementation of algorithm mentioned in the paper (with only one extra BiLSTM layer) with some hyperparameter tuning

import numpy as np
import json
import os
from tqdm import tqdm, trange
import h5py
from prettytable import PrettyTable

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, TimeDistributed, Concatenate
from tensorflow.keras.layers import Attention
from tensorflow.keras import Input, Model

encoder_inputs = Input(shape = (320, 1024))

encoder_BidirectionalLSTM = Bidirectional(LSTM(128, return_sequences = True, return_state = True)) # Encoder Layer
encoder_out, fh, fc, bh, bc = encoder_BidirectionalLSTM(encoder_inputs)
sh = Concatenate()([fh, bh])
ch = Concatenate()([fc, bc])
encoder_states = [sh, ch]

decoder_LSTM = LSTM(256, return_sequences = True, dropout = 0.01, recurrent_dropout = 0.01) # Decoder Layer
decoder_out = decoder_LSTM(encoder_out, initial_state = encoder_states)

attn_layer = Attention(name="Attention_Layer") # Attention Layer
attn_out =  attn_layer([encoder_out, decoder_out])

decoder_concat_input = Concatenate(axis = -1, name = 'concat_layer')([decoder_out, attn_out]) # Concatenating outputs of attention and decoder layer

dense = TimeDistributed(Dense(1, activation = 'sigmoid')) # Output layer
decoder_pred = dense(decoder_concat_input)

model = Model(inputs = encoder_inputs, outputs = decoder_pred) # Model assembling

opt = tf.keras.optimizers.Adam(learning_rate=0.01, beta_1 = 0.8, beta_2 = 0.8) # Adam Optimizer
#opt = tf.keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy']) # Model Compiling
model.summary()
t = trange(10, desc = 'Epoch', ncols = 90)
for epoch_i in t: # Running the model for given epoch numbers

    model.fit(train_loader)
    evaluate_train(epoch_i)
    evaluate_test(epoch_i)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 320, 1024)]  0           []                               
                                                                                                  
 bidirectional (Bidirectional)  [(None, 320, 256),   1180672     ['input_1[0][0]']                
                                 (None, 128),                                                     
                                 (None, 128),                                                     
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                              

Epoch:   0%|                                                       | 0/10 [00:00<?, ?it/s]

8/8 [==============================] - 32s 3s/step - loss: 0.8404 - accuracy: 0.7608



Epoch:   0%|                                                       | 0/10 [00:53<?, ?it/s]

+-------------------------------------------------+
|   Evaluation Result of epoch 0 on Training Set  |
+--------+--------------+------------+------------+
|   ID   |  Precision   |   Recall   |  F-Score   |
+--------+--------------+------------+------------+
|   22   |   0.54216    |  0.71158   |  0.61541   |
|   48   |   0.41737    |  0.55073   |  0.47482   |
|   12   |   0.55779    |  0.73469   |  0.63413   |
|   42   |   0.57713    |  0.76364   |  0.65741   |
|   6    |   0.51808    |  0.68213   |  0.58889   |
|   2    |   0.28081    |  0.37310   |  0.32043   |
|   7    |   0.48801    |  0.64625   |  0.55609   |
|   28   |   0.43660    |  0.57706   |  0.49709   |
|   50   |   0.52640    |  0.69429   |  0.59880   |
|   25   |   0.44176    |  0.58222   |  0.50236   |
|   32   |   0.47255    |  0.61789   |  0.53551   |
|   16   |   0.54835    |  0.72310   |  0.62372   |
|   36   |   0.55629    |  0.73422   |  0.63299   |
|   27   |   0.43949    |  0.58126   |  0.50053   |
|   8    |  

...

+-----------------------------------------------+
|  Evaluation Result of epoch 8 on Testing Set  |
+--------+--------------+-----------+-----------+
|   ID   |  Precision   |   Recall  |  F-Score  |
+--------+--------------+-----------+-----------+
|   11   |   0.56674    |  0.73468  |  0.63982  |
|   14   |   0.48699    |  0.64915  |  0.55648  |
|   31   |   0.52601    |  0.69623  |  0.59925  |
|   47   |   0.42265    |  0.56126  |  0.48218  |
|   19   |   0.50326    |  0.65931  |  0.57081  |
|   1    |   0.61116    |  0.80005  |  0.69296  |
|   41   |   0.72999    |  0.94675  |  0.82433  |
|   13   |   0.40823    |  0.54250  |  0.46585  |
|   30   |   0.53466    |  0.70402  |  0.60773  |
|   9    |   0.55251    |  0.72572  |  0.62738  |
|  mean  |   0.53422    |  0.70197  |  0.60668  |
+--------+--------------+-----------+-----------+
8/8 [==============================] - 24s 3s/step - loss: 0.4825 - accuracy: 0.8122



Epoch:  90%|██████████████████████████████████████████▎    | 9/10 [09:41<01:03, 63.39s/it]

+-------------------------------------------------+
|   Evaluation Result of epoch 9 on Training Set  |
+--------+--------------+------------+------------+
|   ID   |  Precision   |   Recall   |  F-Score   |
+--------+--------------+------------+------------+
|   22   |   0.49014    |  0.64746   |  0.55791   |
|   48   |   0.42180    |  0.56462   |  0.48283   |
|   12   |   0.57163    |  0.75123   |  0.64924   |
|   42   |   0.57713    |  0.76364   |  0.65741   |
|   6    |   0.56577    |  0.74561   |  0.64336   |
|   2    |   0.43415    |  0.57627   |  0.49517   |
|   7    |   0.48801    |  0.64625   |  0.55609   |
|   28   |   0.47688    |  0.63078   |  0.54313   |
|   50   |   0.55007    |  0.72503   |  0.62555   |
|   25   |   0.40825    |  0.53710   |  0.46389   |
|   32   |   0.47469    |  0.62063   |  0.53791   |
|   16   |   0.56872    |  0.75081   |  0.64720   |
|   36   |   0.55178    |  0.73065   |  0.62874   |
|   27   |   0.38196    |  0.50488   |  0.43490   |
|   8    |  


Epoch: 100%|██████████████████████████████████████████████| 10/10 [09:45<00:00, 58.58s/it]

+-----------------------------------------------+
|  Evaluation Result of epoch 9 on Testing Set  |
+--------+--------------+-----------+-----------+
|   ID   |  Precision   |   Recall  |  F-Score  |
+--------+--------------+-----------+-----------+
|   11   |   0.52871    |  0.70680  |  0.60486  |
|   14   |   0.53694    |  0.71282  |  0.61249  |
|   31   |   0.52519    |  0.69168  |  0.59703  |
|   47   |   0.47790    |  0.63659  |  0.54593  |
|   19   |   0.51802    |  0.68648  |  0.59046  |
|   1    |   0.60575    |  0.79409  |  0.68725  |
|   41   |   0.72999    |  0.94675  |  0.82433  |
|   13   |   0.48100    |  0.63434  |  0.54710  |
|   30   |   0.53466    |  0.70402  |  0.60773  |
|   9    |   0.56909    |  0.75181  |  0.64781  |
|  mean  |   0.55073    |  0.72654  |  0.62650  |
+--------+--------------+-----------+-----------+


In [ ]:
# Improved with two extra BiLSTM layers, one extra LSTM layer and two more Time Distributed layers and some hyperparameter tuning

import numpy as np
import json
import os
from tqdm import tqdm, trange
import h5py
from prettytable import PrettyTable

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, TimeDistributed, Concatenate
from tensorflow.keras.layers import Attention
from tensorflow.keras import Input, Model
from keras.layers import LeakyReLU

encoder_inputs = Input(shape = (320, 1024))

# Encoder Layers
encoder_BidirectionalLSTM = Bidirectional(LSTM(64, return_sequences = True, return_state = True))
encoder_BidirectionalLSTM1 = Bidirectional(LSTM(64, return_sequences = True, return_state = True))
encoder_BidirectionalLSTM2 = Bidirectional(LSTM(64, return_sequences = True, return_state = True))
encoder_out, fh, fc, bh, bc = encoder_BidirectionalLSTM(encoder_inputs)
encoder_out, fh, fc, bh, bc = encoder_BidirectionalLSTM1(encoder_out)
encoder_out, fh, fc, bh, bc = encoder_BidirectionalLSTM2(encoder_out)
sh = Concatenate()([fh, bh])
ch = Concatenate()([fc, bc])
encoder_states = [sh, ch]

# Decoder Layers
decoder_LSTM = LSTM(128, return_sequences = True)
decoder_out = decoder_LSTM(encoder_out, initial_state = encoder_states)
decoder_out = decoder_LSTM(decoder_out)

# Attention Layer
attn_layer = Attention(name="Attention_Layer")
attn_out =  attn_layer([encoder_out, decoder_out])

# Concatenating decoder and attention outputs
decoder_concat_input = Concatenate(axis = -1, name = 'concat_layer')([decoder_out, attn_out])

# Time Distributed Dense Layer (for multi-dimensional) with relu activation function
dense = TimeDistributed(Dense(42, activation = 'relu'))
decoder_pred = dense(decoder_concat_input)

# Time Distributed Dense Layer (for multi-dimensional) with tanh activation function
dense = TimeDistributed(Dense(14, activation = 'tanh'))
decoder_pred = dense(decoder_pred)

# Time Distributed Dense Layer (for multi-dimensional) with sigmoid activation function (Output Layer)
dense = TimeDistributed(Dense(1, activation = 'sigmoid'))
decoder_pred = dense(decoder_pred)

# Model assembling
model = Model(inputs = encoder_inputs, outputs = decoder_pred)

# Optimizers
opt = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1 = 0.8, beta_2 = 0.8)
#opt = tf.keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

# Compiling the model
model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy'])
model.summary()
t = trange(10, desc = 'Epoch', ncols = 90)
for epoch_i in t: # Running the model for given epoch numbers

    model.fit(train_loader)
    evaluate_train(epoch_i)
    evaluate_test(epoch_i)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 320, 1024)]  0           []                               
                                                                                                  
 bidirectional (Bidirectional)  [(None, 320, 128),   557568      ['input_1[0][0]']                
                                 (None, 64),                                                      
                                 (None, 64),                                                      
                                 (None, 64),                                                      
                                 (None, 64)]                                                      
                                                                                              

Epoch:   0%|                                                       | 0/10 [00:00<?, ?it/s]

8/8 [==============================] - 27s 2s/step - loss: 0.5199 - accuracy: 0.8114



Epoch:   0%|                                                       | 0/10 [00:51<?, ?it/s]

+-------------------------------------------------+
|   Evaluation Result of epoch 0 on Training Set  |
+--------+--------------+------------+------------+
|   ID   |  Precision   |   Recall   |  F-Score   |
+--------+--------------+------------+------------+
|   22   |   0.52554    |  0.69479   |  0.59841   |
|   48   |   0.40163    |  0.54033   |  0.46073   |
|   12   |   0.51695    |  0.67991   |  0.58734   |
|   42   |   0.61093    |  0.80145   |  0.69333   |
|   6    |   0.55246    |  0.72927   |  0.62867   |
|   2    |   0.39443    |  0.52540   |  0.45055   |
|   7    |   0.50584    |  0.65785   |  0.57192   |
|   28   |   0.46792    |  0.61780   |  0.53251   |
|   50   |   0.48618    |  0.63790   |  0.55180   |
|   25   |   0.40973    |  0.54073   |  0.46620   |
|   32   |   0.47255    |  0.61789   |  0.53551   |
|   16   |   0.55618    |  0.73344   |  0.63262   |
|   36   |   0.55892    |  0.73960   |  0.63669   |
|   27   |   0.40190    |  0.53154   |  0.45771   |
|   8    |  

...

+-----------------------------------------------+
|  Evaluation Result of epoch 8 on Testing Set  |
+--------+--------------+-----------+-----------+
|   ID   |  Precision   |   Recall  |  F-Score  |
+--------+--------------+-----------+-----------+
|   11   |   0.55124    |  0.73912  |  0.63145  |
|   14   |   0.45699    |  0.60557  |  0.52088  |
|   31   |   0.49835    |  0.65769  |  0.56702  |
|   47   |   0.41109    |  0.55238  |  0.47136  |
|   19   |   0.52567    |  0.68426  |  0.59457  |
|   1    |   0.61294    |  0.80665  |  0.69658  |
|   41   |   0.70431    |  0.92829  |  0.80091  |
|   13   |   0.48100    |  0.63434  |  0.54710  |
|   30   |   0.53466    |  0.70402  |  0.60773  |
|   9    |   0.51901    |  0.68469  |  0.59045  |
|  mean  |   0.52953    |  0.69970  |  0.60280  |
+--------+--------------+-----------+-----------+
8/8 [==============================] - 12s 2s/step - loss: 0.4828 - accuracy: 0.8122



Epoch:  90%|██████████████████████████████████████████▎    | 9/10 [06:17<00:37, 37.52s/it]

+-------------------------------------------------+
|   Evaluation Result of epoch 9 on Training Set  |
+--------+--------------+------------+------------+
|   ID   |  Precision   |   Recall   |  F-Score   |
+--------+--------------+------------+------------+
|   22   |   0.52035    |  0.68836   |  0.59266   |
|   48   |   0.42180    |  0.56462   |  0.48283   |
|   12   |   0.55779    |  0.73469   |  0.63413   |
|   42   |   0.61093    |  0.80145   |  0.69333   |
|   6    |   0.53950    |  0.70765   |  0.61224   |
|   2    |   0.45104    |  0.59682   |  0.51373   |
|   7    |   0.46588    |  0.60929   |  0.52802   |
|   28   |   0.46872    |  0.61997   |  0.53383   |
|   50   |   0.54127    |  0.71555   |  0.61633   |
|   25   |   0.42494    |  0.56336   |  0.48446   |
|   32   |   0.51539    |  0.65861   |  0.57823   |
|   16   |   0.56947    |  0.75059   |  0.64760   |
|   36   |   0.56424    |  0.74720   |  0.64295   |
|   27   |   0.42845    |  0.56609   |  0.48775   |
|   8    |  


Epoch: 100%|██████████████████████████████████████████████| 10/10 [06:20<00:00, 38.05s/it]

+-----------------------------------------------+
|  Evaluation Result of epoch 9 on Testing Set  |
+--------+--------------+-----------+-----------+
|   ID   |  Precision   |   Recall  |  F-Score  |
+--------+--------------+-----------+-----------+
|   11   |   0.55309    |  0.73816  |  0.63231  |
|   14   |   0.51498    |  0.68712  |  0.58871  |
|   31   |   0.60184    |  0.79183  |  0.68387  |
|   47   |   0.47692    |  0.62201  |  0.53987  |
|   19   |   0.50570    |  0.67015  |  0.57642  |
|   1    |   0.59394    |  0.77563  |  0.67273  |
|   41   |   0.69956    |  0.92463  |  0.79647  |
|   13   |   0.48100    |  0.63434  |  0.54710  |
|   30   |   0.53466    |  0.70402  |  0.60773  |
|   9    |   0.51901    |  0.68469  |  0.59045  |
|  mean  |   0.54807    |  0.72326  |  0.62357  |
+--------+--------------+-----------+-----------+


# **Getting an Output :**

In [ ]:
# Reading the h5 file
data_file = h5py.File('/content/drive/MyDrive/suvidha internship/Resources/Submission/fcsn_tvsum.h5')

In [ ]:
# Predicting scores for a particular video using the model
pred_score = model.predict(np.array(data_file['video_30']['feature']).reshape(-1,320,1024))
video_info = data_file['video_35']
pred_score, pred_selected, pred_summary = select_keyshots(video_info, pred_score)

In [ ]:
# Selected shots
pred_selected

[6, 7, 12, 13, 14, 15, 16, 17, 19, 22, 24, 28]

In [ ]:
# Getting the output summary video
import cv2
cps = video_info['change_points'][()]

video = cv2.VideoCapture('/content/drive/MyDrive/suvidha internship/Resources/Submission/video/video_35.mp4')
frames = []
success, frame = video.read()
while success:
    frames.append(frame)
    success, frame = video.read()
frames = np.array(frames)
keyshots = []
for sel in pred_selected:
    for i in range(cps[sel][0], cps[sel][1]):
         keyshots.append(frames[i])
keyshots = np.array(keyshots)

video_writer = cv2.VideoWriter('summary.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 24, keyshots.shape[2:0:-1])
for frame in keyshots:
    video_writer.write(frame)
video_writer.release()